In [5]:
import pandas as pd
pd.options.plotting.backend = "plotly"

In [6]:
filepath = "records_world.json"
df = pd.read_json(filepath, orient="records")
n_years_back = 24

# We set the index to the date
df.set_index("name", inplace=True)
df = df.join(df["data"].apply(pd.Series))
df.drop(columns=["data"], inplace=True)
df = df.iloc[:-3]
df = df.T
df = df.drop("1981", axis=1)

In [7]:
# all columns where its not leap year we insert a NaN value at index 31 + 28 = 59 and shift the rest of the values one position to the right
for col in df.columns:
    if int(col) % 4 != 0:
        df[col][60:] = df[col][59:-1].values
        df[col][59] = None

/var/folders/ww/bqf8vs0973j5m8pvqkq5_j3h0000gn/T/ipykernel_27522/1339617148.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[col][60:] = df[col][59:-1].values
/var/folders/ww/bqf8vs0973j5m8pvqkq5_j3h0000gn/T/ipykernel_27522/1339617148.

In [8]:
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors

n_years_back = len(df.columns[1:])
grey_colors = n_colors('rgb(145, 100, 100)', 'rgb(180, 180, 180)', n_years_back -2, colortype='rgb')
red_colors = n_colors('rgb(220, 100, 100)', 'rgb(220, 50, 50)', 2, colortype='rgb')
colors = grey_colors[::-1] + red_colors


# Create the plot
fig_lines = go.Figure()

# Add traces for each year
for i, column in enumerate(df.columns[1:]):
    fig_lines.add_trace(go.Scatter(x=df.index, y=df[column], mode='lines', name=str(column),
                             line=dict(color=colors[i], width=1.5)))

# Update layout
fig_lines.update_layout(title='Yearly Trends',
                  xaxis_title='Day of year',
                  yaxis_title='Value')

fig_lines.update_yaxes(title_text="Temperature (°C)")
fig_lines.update_layout(title="Daily Sea Surface Temperature (°C) World (60°S-60°N, 0-360°E)")

# Show plot
fig_lines.show()


# We begin to look at the deltas

In [9]:
df_diff = df.T.diff()
df_diff_rolling = df_diff.rolling(1).mean()

# We keep only the 10 last columns
df_diff_rolling = df_diff_rolling.iloc[-n_years_back:]

In [10]:
fig_boxplots = go.Figure()

# Calculate colors
n_years_back = len(df_diff_rolling.T.columns)
grey_colors = n_colors('rgb(145, 100, 100)', 'rgb(180, 180, 180)', n_years_back - 2, colortype='rgb')
red_colors = n_colors('rgb(220, 100, 100)', 'rgb(220, 50, 50)', 2, colortype='rgb')
colors = grey_colors[::-1] + red_colors

# Add box plots for each year
for i, column in enumerate(df_diff_rolling.T.columns):
    fig_boxplots.add_trace(go.Box(y=df_diff_rolling.T[column], name=str(column), marker_color=colors[i]))

# Update layout
fig_boxplots.update_layout(
    title='Boxplots for Each Year',
    xaxis_title='Year',
    yaxis_title='Values',
)

fig_boxplots.update_yaxes(title_text="Daily temperature difference (°C) from previous year")
fig_boxplots.update_layout(showlegend=False)
fig_boxplots.update_layout(title="Change in Daily Sea Surface Temperature (°C) from previous year")

# Show plot
fig_boxplots.show()

In [11]:
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

# Generate grey color for the first traces
#grey_colors = n_colors('rgb(245, 200, 200)','rgb(220, 220, 220)', n_years_back, colortype='rgb')
grey_colors = n_colors('rgb(145, 100, 100)', 'rgb(180, 180, 180)', n_years_back, colortype='rgb')
red_colors = n_colors('rgb(220, 100, 100)', 'rgb(220, 50, 50)', 2, colortype='rgb')
colors = grey_colors[::-1] + red_colors
colors = colors[::-1]

# We reverse the order of the rows in df_diff_rolling to have the most recent year at the top
df_diff_rolling_plot = df_diff_rolling.iloc[::-1]

fig_dist = go.Figure()
for data_line, color in zip(df_diff_rolling_plot.values, colors):
    fig_dist.add_trace(go.Violin(x=data_line, line_color=color))

fig_dist.update_traces(orientation='h', side='positive', width=3, points=False)
fig_dist.update_layout(xaxis_showgrid=False, xaxis_zeroline=False)

fig_dist.update_layout(yaxis=dict(tickmode='array', tickvals=np.arange(n_years_back), ticktext=df_diff_rolling_plot.index))
fig_dist.add_shape(type="line", x0=0, y0=0, x1=0, y1=n_years_back, line=dict(color="black", width=1, dash="dot"))
fig_dist.update_xaxes(title_text="Daily temperature difference (°C) from previous year")
fig_dist.update_layout(showlegend=False)
fig_dist.update_layout(title="Distributions of change in Daily Sea Surface Temperature (°C) from previous year")
fig_dist.update_layout(height=1200, width=800)
fig_dist.show()

# We combine the plots

In [12]:
from plotly.subplots import make_subplots

# Create subplot figure
fig_combined1 = make_subplots(rows=2, cols=1, vertical_spacing=0.1)

# Add fig1 to the first subplot
for trace in fig_lines['data']:
    fig_combined1.add_trace(trace, row=1, col=1)

# Add x-axis title to fig1 trace
fig_combined1.update_xaxes(title_text="Day of the year", row=1, col=1)
fig_combined1.update_yaxes(title_text="Temperature (°C)", row=1, col=1)

# Add fig2 to the second subplot
for trace in fig_boxplots['data']:
    fig_combined1.add_trace(trace, row=2, col=1)

fig_combined1.update_xaxes(title_text="Year", row=2, col=1)
fig_combined1.update_yaxes(title_text="Daily temperature <br> difference (°C) from <br> previous year", row=2, col=1)

# Update layout
fig_combined1.update_layout(title_text="Daily Sea Surface Temperature (°C) World (60°S-60°N, 0-360°E)")
# We add text below the title
fig_combined1.add_annotation(
    x=-0.045,
    y=1.07,
    xref="paper",
    yref="paper",
    text="Data source: Climate Reanalyzer (n.d.). [Daily Sea Surface Temperature]. Climate Change Institute, University of Maine. Retrieved [03 17, 2024], from https://climatereanalyzer.org/",
    showarrow=False,
)
fig_combined1.update_layout(height=800, width=1200)

# We reverse the order of the legend
fig_combined1.update_layout(legend=dict(traceorder="reversed"))

# Show plot
fig_combined1.show()

# We export the combined plot both to html and png
fig_combined1.write_html("combined_plot.html")
fig_combined1.write_image("combined_plot.png", scale=4)